In [1]:
import cv2


import os
import random
from rasterio.features import shapes
import matplotlib.pyplot as plt
import numpy as np
import rasterio

from osgeo import ogr, gdal
from torchvision.transforms import transforms
from tqdm import tqdm
from shapely.geometry import polygon
from utils import generate_patch_coordinates



KeyboardInterrupt: 

In [ ]:
dataset_dir = os.path.join('C:','Users','bhatt','OneDrive','Desktop','butwal data')
image_dir = "C:\\Users\\bhatt\\OneDrive\\Desktop\\butwal data\\Orthomosaics Output\\Butwal Orthomosaics Only"
mask_dir = "C:\\Users\\bhatt\\OneDrive\\Documents\\ArcGIS\\Projects\\MyProject33"
current_image_file_name = 'Mission 2.tif'
current_mask_file_name = 'Buildings.shp'

# Path to the image file in .tif format
image_path = os.path.join(image_dir, current_image_file_name)
# Path to the shape file containing the landcover classes in .shp format
mask_path = os.path.join(mask_dir,
                         current_mask_file_name)
# Path to the shape file containing the landcover classes in .shp format
# mask_path = os.path.join(mask_dir,
#                          current_mask_file_name)
output12_dir = os.path.join('.', 'output12')
os.makedirs(output12_dir, exist_ok=True)

rasterized_dir = os.path.join(output12_dir, 'rasterized_outputs')
os.makedirs(rasterized_dir, exist_ok=True)
# rasterized_dir = os.path.join(output_dir, 'rasterized_outputs')
# os.makedirs(rasterized_dir, exist_ok=True)
rasterized_file_name = "Result12.tif"
rasterized_file_path = os.path.join(rasterized_dir, rasterized_file_name)
patch_size = 513
stride = 256
boundary = [3053710.461731, 751158.377075, 3055429.699707, 753175.112122]
patch_output_dir = os.path.join(output12_dir, f"{patch_size}x{patch_size}")
os.makedirs(patch_output_dir, exist_ok=True)
labels_output_dir = os.path.join(patch_output_dir, 'labels')
os.makedirs(labels_output_dir, exist_ok=True)




In [ ]:
mask_shape_file_path = mask_path
image_file_path = image_path

gdalformat = 'GTiff'
datatype = gdal.GDT_Float32

# Get projection info from reference image
Image = gdal.Open(image_file_path, gdal.GA_ReadOnly)

# Open Shapefile
Shapefile = ogr.Open(mask_shape_file_path)
Shapefile_layer = Shapefile.GetLayer()

# Rasterise
print("Rasterizing shapefile...")

Output = gdal.GetDriverByName(gdalformat).Create(rasterized_file_path, Image.RasterXSize, Image.RasterYSize, 1,
                                                 datatype,
                                                 options=['COMPRESS=DEFLATE'])
Output.SetProjection(Image.GetProjectionRef())
Output.SetGeoTransform(Image.GetGeoTransform())

# Write data to band 1
Band = Output.GetRasterBand(1)
Band.SetNoDataValue(0)

gdal.RasterizeLayer(Output, [1], Shapefile_layer, options=['ATTRIBUTE=Class_id'])

# Close datasets
Band = None
Output = None
Image = None
Shapefile = None

print("Rasterization of shapefile completed.")

In [ ]:
# Open the rasters
with rasterio.open(image_path) as raster1:
    temp_img = raster1.read()  # Read raster1 into numpy array

with rasterio.open(rasterized_file_path) as raster2:
    temp_mask = raster2.read(1)  # Read raster2 into numpy array

labels, count = np.unique(temp_mask, return_counts=True)  #Check for each channel. All channels are identical
print('Unique values in mask: ', labels)
print('Counts of unique values: ', count)

In [ ]:
with rasterio.open(rasterized_file_path, "r") as src:
    mask_data = src.read()
    mask_meta = src.meta

print(np.unique(mask_data))

In [ ]:
from utils import create_patches
# image_path="C:\\Users\\bhatt\\OneDrive\Desktop\\butwal data\\Orthomosaics Output\\Butwal Orthomosaics Only\\Mission 1.tif"
# rasterized_file_path="C:\\Users\\bhatt\\OneDrive\\Desktop\\semprj\\aerial-satellite-imagery-segmentation-nepal\\output11\\rasterized_outputs\\Result11.tif"
# patch_size= 513
# stride=256
# patch_output_dir="C:\\Users\\bhatt\\OneDrive\\Desktop\\semprj\\aerial-satellite-imagery-segmentation-nepal\\output11\\513x513"
create_patches(image_path, rasterized_file_path, patch_output_dir, patch_size, stride)

In [ ]:
# patch_output_dir=r"C:\Users\bhatt\OneDrive\Desktop\semprj\aerial-satellite-imagery-segmentation-nepal\output12\513x513"
image_files = [f for f in os.listdir(patch_output_dir + "\\images") if f.endswith('.jpg')]

# Select a random image file
random_image_file = random.choice(image_files)

# Construct the full paths to the image and mask files
temp_image_path = os.path.join(patch_output_dir, "images", random_image_file)
temp_mask_path = os.path.join(patch_output_dir, "masks", random_image_file.replace('.jpg', '_mask.jpg'))

# Open the image and mask files using Rasterio
with rasterio.open(temp_image_path) as src:
    image = src.read().astype(float)
    image_transform = src.transform

with rasterio.open(temp_mask_path) as src:
    mask = src.read()
    mask_transform = src.transform

# The image data read by Rasterio is in (bands, rows, cols) order
# Convert the image data to (rows, cols, bands) order for visualization
transposed_image = image.transpose((1, 2, 0))

if 0 not in np.unique(transposed_image):
    # Scale the image data to be between 0 and 1 for better visualization
    transposed_image -= transposed_image.min()
    transposed_image /= transposed_image.max()

    # Create plots
    fig, ax = plt.subplots(1, 2, figsize=(15, 15))

    # Display the image
    ax[0].imshow(transposed_image)
    ax[0].set_title("Image")

    # Display the mask
    transposed_mask = mask.transpose((1, 2, 0))

    ax[1].imshow(transposed_mask)
    ax[1].set_title("Mask")
    print(np.unique(mask))

    plt.tight_layout()

    plt.show()


    print('Image shape: ', image.shape)
    print('Mask shape: ', mask.shape)
else:
    print("Blank data")

In [ ]:
import matplotlib.pyplot as plt
import os
import random
import rasterio
import numpy as np


# Get list of image files
image_files = [f for f in os.listdir(patch_output_dir + "\\images") if f.endswith('.jpg')]
print(image_files)
# Select a random image file
random_image_file = random.choice(image_files)

# Construct the full paths to the image and mask files
temp_image_path = os.path.join(patch_output_dir, "images", random_image_file)
temp_mask_path = os.path.join(patch_output_dir, "masks", random_image_file.replace('.jpg', '_mask.jpg'))

# Open the image and mask files using Rasterio
with rasterio.open(temp_image_path) as src:
    image = src.read().astype(float)
    image_transform = src.transform
# temp_mask_path=r"C:\Users\bhatt\OneDrive\Desktop\semprj\aerial-satellite-imagery-segmentation-nepal\Output_mg\513x513\masks\test\Mission4patch_955_mask.jpg"

with rasterio.open(temp_mask_path) as src:
    mask = src.read()
    mask_transform = src.transform
print(np.unique(mask))
# The image data read by Rasterio is in (bands, rows, cols) order
# Convert the image data to (rows, cols, bands) order for visualization
transposed_image = image.transpose((1, 2, 0))

# Scale the image data to be between 0 and 1 for better visualization
transposed_image -= transposed_image.min()
transposed_image /= transposed_image.max()

# Create plots
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Display the image
ax[0].imshow(transposed_image)
ax[0].set_title("Image")

# Display the first channel of the mask
ax[1].imshow(mask[0,:,:], cmap='gray')
ax[1].set_title("Mask Channel 1")

# Display the second channel of the mask
ax[2].imshow(mask[1,:,:], cmap='gray')
ax[2].set_title("Mask Channel 2")

plt.tight_layout()
plt.show()
# Create plots
# fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# # Combine the channels into a single image
# combined_mask = np.stack((mask[0,:,:], mask[1,:,:], np.zeros_like(mask[0,:,:])), axis=-1)

# # Display the combined mask
# ax.imshow(combined_mask)
# ax.set_title("Combined Mask")

# plt.tight_layout()
# plt.show()



print('Image shape:', image.shape)
print('Mask shape:', mask.shape)


In [ ]:
print(patch_output_dir)

In [ ]:
from generator2 import NepalDataGenerator
from generator2 import NepalDataset
from utils import visualize

In [ ]:
data_path = patch_output_dir
in_channels = 4
num_classes = 4
batch_size = 4
shuffle = True

transform = transforms.Compose([
    transforms.ToTensor(),
    
])

dataset = NepalDataset(data_path, transform=transform)
print(f"Dataset size: {format(len(dataset))}")
data_generator = NepalDataGenerator(dataset, batch_size=batch_size, shuffle=shuffle)
print(f"Data generator size: {format(len(data_generator))}")

In [ ]:
no_of_images_to_show = 4 # Number of images to show from the batch

images, masks = data_generator.__next__()  # Get the batch of images and masks

if no_of_images_to_show > batch_size:
    no_of_images_to_show = batch_size

for i in range(0, no_of_images_to_show):
    image = images[i].permute(1, 2, 0).numpy()  # Access individual image and convert to numpy array
    mask = masks[i].squeeze().numpy()  # Access individual mask and convert to numpy array
    visualize(image, mask)

YOLO V8 preparation


In [ ]:
import numpy as np
import cv2
from shapely.geometry import Polygon
import tifffile


def mask_to_polygons(img_path, mask_path):
    '''
    Convierte una máscara de imagen en polígonos. Devuelve dos listas:
    - Lista de polígonos de shapely sin normalizar
    - Lista de polígonos de shapely normalizados (coordenadas entre 0 y 1)

    Args:
        img_path (str): Ruta al archivo de imagen original.
        mask_path (str): Ruta al archivo de la máscara en escala de grises.
    '''

    mask =tifffile.imread(mask_path)

    # mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Calcula los contornos 
    mask = mask.astype(bool)
    #contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # convertimos los contornos a polígonos de Label Studio
    polygons = []
    normalized_polygons = []
    for contour in contours:

        # Lo meto en un try porque la extraccion de polígonos que hace el opencv a partir de la máscara
        # a veces genera polígonos de menos de 4 vértices, que no tiene sentido por no ser cerrados, 
        # provocando que falle al convertir a polígno de shapely

        try:
            polygon = contour.reshape(-1, 2).tolist()

            # normalizamos las coordenadas entre 0 y 1 porque así lo requiere YOLOv8
            normalized_polygon = [[round(coord[0] / mask.shape[1], 4), round(coord[1] / mask.shape[0], 4)] for coord in
                                  polygon]

            # Convertimos a objeto poligono de shapely (sin normalizar)
            polygon_shapely = Polygon(polygon)
            simplified_polygon = polygon_shapely.simplify(0.85, preserve_topology=True)
            polygons.append(simplified_polygon)

            # normalizdos
            normalized_polygons.append(Polygon(normalized_polygon))


        except Exception as e:
            pass

    return polygons, normalized_polygons

In [ ]:
def mask_to_polygons_per_class(img_path, mask_path):
    # Map grayscale mask values to your class indices
    class_mapping = {0: 0, 1: 1,2:2}

    mask = tifffile.imread(mask_path)

    polygons_per_class = {}
    for mask_value in np.unique(mask):
        # Look up class index using mask value, if no mapping is found then continue
        class_index = class_mapping.get(mask_value)
        if class_index is None:
            continue

        class_mask = np.where(mask == mask_value, 1, 0).astype(np.uint8)

        contours, _ = cv2.findContours(class_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


        polygons = []
        for contour in contours:
            try:
                polygon = contour.reshape(-1, 2).tolist()

                normalized_polygon = [
                    
                    [round(coord[0] / mask.shape[1], 4), round(coord[1] / mask.shape[0], 4)]
                    for coord in polygon
                ]

                if class_index not in polygons_per_class:
                    polygons_per_class[class_index] = []
                polygons_per_class[class_index].append(Polygon(normalized_polygon))

            except Exception as e:
                pass

    return polygons_per_class


In [ ]:
import os
import cv2

input_dir = os.path.join(patch_output_dir, 'masks')
output_dir = labels_output_dir

for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    polygons, normalized_polygons = mask_to_polygons(image_path, image_path)  # Separate lists for each class

    # print the polygons
    file_name_without_ext = os.path.splitext(j)[0]
    if file_name_without_ext.endswith('_mask'):
        file_name_without_ext = file_name_without_ext[:-5]

    with open('{}.txt'.format(os.path.join(output_dir, file_name_without_ext)), 'w') as f:
        for class_label in range(3):
            for polygon in normalized_polygons:
                f.write('{} '.format(class_label))  # Add class label at the beginning of each line
                if isinstance(polygon, Polygon):
                    x, y = polygon.exterior.xy
                    for i in range(len(x)):
                        f.write('{} {}\t'.format(x[i], y[i]))
                f.write('\n')


In [ ]:
import os
import cv2

input_dir = os.path.join(patch_output_dir, 'masks')
output_dir = labels_output_dir


for j in os.listdir(input_dir):
    image_path = os.path.join(input_dir, j)
    polygons_per_class = mask_to_polygons_per_class(image_path, image_path)

    # print the polygons
    file_name_without_ext = os.path.splitext(j)[0]
    if file_name_without_ext.endswith('_mask'):
        file_name_without_ext = file_name_without_ext[:-5]
    
    with open('{}.txt'.format(os.path.join(output_dir, file_name_without_ext)), 'w') as f:
        for class_label, polygons in polygons_per_class.items():
            for polygon in polygons:
                f.write('{} '.format(class_label))  # Add class label at the beginning of each line
                x, y = polygon.exterior.xy
                for i in range(len(x)):
                    f.write('{} {}\t'.format(x[i], y[i]))
                f.write('\n')

In [ ]:
# import os
# import shutil
# import numpy as np


# def split_dataset_into_train_val_test(input_folder, output_folder, ratio=(0.7, 0.15, 0.15)):
#     assert sum(ratio) == 1, "Ratios must add up to 1."

#     # Get all file names in the input folder
#     all_files = os.listdir(input_folder)
#     np.random.shuffle(all_files)

#     num_files = len(all_files)
#     train_files = all_files[:int(num_files * ratio[0])]
#     val_files = all_files[int(num_files * ratio[0]):int(num_files * (ratio[0] + ratio[1]))]
#     test_files = all_files[int(num_files * (ratio[0] + ratio[1])):]

#     # Create output directories
#     for dir in ['train', 'val', 'test']:
#         os.makedirs(os.path.join(output_folder, dir), exist_ok=True)

#     # Move files to respective directories
#     for file in train_files:
#         shutil.move(os.path.join(input_folder, file), os.path.join(output_folder, 'train', file))

#     for file in val_files:
#         shutil.move(os.path.join(input_folder, file), os.path.join(output_folder, 'val', file))

#     for file in test_files:
#         shutil.move(os.path.join(input_folder, file), os.path.join(output_folder, 'test', file))


# directories = ['images', 'labels', 'masks']
# patch_output_directory = patch_output_dir

# for directory in directories:
#     input_folder = os.path.join(patch_output_directory, directory)
#     output_folder = os.path.join(patch_output_directory, directory)
#     split_dataset_into_train_val_test(input_folder, output_folder)

In [ ]:
import os
import shutil
import numpy as np
def split_dataset_into_train_val_test(input_folder_images, input_folder_masks, output_folder, ratio=(0.7, 0.15, 0.15)):
    assert sum(ratio) == 1, "Ratios must add up to 1."
    # Get all image file names in the input folder (tif)
    all_image_files = [f for f in os.listdir(input_folder_images) if f.endswith('.tif')]
    np.random.shuffle(all_image_files)
    num_files = len(all_image_files)
    train_files = all_image_files[:int(num_files * ratio[0])]
    val_files = all_image_files[int(num_files * ratio[0]):int(num_files * (ratio[0] + ratio[1]))]
    test_files = all_image_files[int(num_files * (ratio[0] + ratio[1])):]
    # Create output directories
    for dir in ['train', 'val', 'test']:
        os.makedirs(os.path.join(output_folder, 'images', dir), exist_ok=True)
        os.makedirs(os.path.join(output_folder, 'masks', dir), exist_ok=True)
    # Function to move files
    def move_files(files, subset):
        for file in files:
            base_name = os.path.splitext(file)[0]
            image_file = file
            mask_file = base_name + '_mask.tif'  # Updated for tif
            src_image_path = os.path.join(input_folder_images, image_file)
            src_mask_path = os.path.join(input_folder_masks, mask_file)
            dest_image_path = os.path.join(output_folder, 'images', subset, image_file)
            dest_mask_path = os.path.join(output_folder, 'masks', subset, mask_file)
            # Check if the mask file exists before moving
            if os.path.exists(src_mask_path):
                shutil.move(src_image_path, dest_image_path)
                shutil.move(src_mask_path, dest_mask_path)
            else:
                print(f"Warning: Mask file {src_mask_path} not found for image file {src_image_path}. Skipping.")
    move_files(train_files, 'train')
    move_files(val_files, 'val')
    move_files(test_files, 'test')
# Directories
patch_output_directory = patch_output_dir  # your patches folder
input_folder_images = os.path.join(patch_output_directory, 'images')
input_folder_masks = os.path.join(patch_output_directory, 'masks')
split_dataset_into_train_val_test(input_folder_images, input_folder_masks, patch_output_directory)


In [ ]:
# import shutil
# import os


# def merge_all_files(directory):
#     main_folders = ['images', 'labels', 'masks']
#     subfolders = ['test', 'train', 'val']

#     for main_folder in main_folders:
#         main_folder_path = os.path.join(directory, main_folder)

#         for subfolder in subfolders:
#             subfolder_path = os.path.join(main_folder_path, subfolder)

#             # merge all files in subfolder to main_folder
#             for filename in os.listdir(subfolder_path):
#                 shutil.move(os.path.join(subfolder_path, filename), main_folder_path)

#             # remove subfolder
#             shutil.rmtree(subfolder_path)
# # function call
# merge_all_files(patch_output_dir)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8m-seg.pt')

model.train(data='config.yaml', epochs=100, imgsz=512, single_cls=False,patience=30,optimizer='Adam')

In [ ]:
metrics=model.val()
# 


In [ ]:
import cv2
from ultralytics import YOLO

model_path = "C:\\Users\\bhatt\\runs\\segment\\train10\\weights\\best.pt"

image_path = "C:\\Users\\bhatt\\OneDrive\\Desktop\\semprj\\aerial-satellite-imagery-segmentation-nepal\\output\\512x512\\images\\test\\patch_1700.jpg"

img= cv2.imread(image_path)
H, W, _ = img.shape

model = YOLO(model_path)

results = model(img)

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.show()  # display to screen
    result.save(filename='result.jpg')  # save to disk
# 
# for result in results:
#     for j,mask in enumerate(result.masks.data):
#         mask=mask.cpu().numpy()*255
#         mask=cv2.resize(mask, (W,H))
#         visualize(img,mask)
#         print(f"Class of mask {j}: {result.masks.data.names[j]}")

In [ ]:
# import os
# import matplotlib.pyplot as plt
# import tifffile
# patch_output_dir = r'output\512x512'
# input_dir = os.path.join(patch_output_dir, 'masks')
# # output_dir = labels_output_dir

# for j in os.listdir(input_dir):
#     print(j)
#     image_path = os.path.join(input_dir, j)
#     # mask = tifffile.imread(image_path)
#     plt.imread(r"output\512x512\masks\test\patch_381_mask.jpg")
#     plt.show()
#     # image_path = os.path.join(input_dir, j)
#     # polygons, normalized_polygons = mask_to_polygons(image_path, image_path)